<a href="https://colab.research.google.com/github/NickKornienko/Clustering/blob/main/Audio_embeddings_using_LLMs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install librosa transformers sklearn

  Preparing metadata (setup.py) ... done
  Created wheel for sklearn: filename=sklearn-0.0.post11-py3-none-any.whl size=2959 sha256=c8b90715af5efd041f90e40d279c8f2c4a9bceb209b86fc5c4179da0a926da6a
  Stored in directory: /root/.cache/pip/wheels/aa/9c/60/f67813603a52fc35057868f1aba0003cc75b72583dcaa2c341
Successfully built sklearn


In [3]:
import librosa
import numpy as np
import torch
from transformers import AutoModel, AutoTokenizer
from sklearn.cluster import KMeans
import os
from scipy.spatial.distance import cdist

In [4]:
def load_and_extract_features(file_path):
    y, sr = librosa.load(file_path, sr=None)
    mfcc = librosa.feature.mfcc(y=y, sr=sr)
    return mfcc.flatten()

# Path to your audio files
audio_folder_path = '/content/audio_files/'

# List all audio files
audio_files = [f for f in os.listdir(audio_folder_path) if f.endswith('.wav')]

# Load and extract features from each audio file
audio_features = [load_and_extract_features(os.path.join(audio_folder_path, file)) for file in audio_files]

In [5]:
# Load a pre-trained transformer model
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModel.from_pretrained("bert-base-uncased")

# Function to get embeddings from the model
def get_embeddings(features):
    inputs = tokenizer(str(features.tolist()), return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).numpy()

# Get embeddings for each audio file
embeddings = np.array([get_embeddings(features) for features in audio_features])

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [6]:
# Number of clusters
n_clusters = 5

# Perform clustering
kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(embeddings.squeeze())

# Get cluster labels
labels = kmeans.labels_

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [7]:
# Analyze cluster labels
for file, label in zip(audio_files, labels):
    print(f"{file}: Cluster {label}")

Major_12.wav: Cluster 2
Minor_18.wav: Cluster 4
Major_11.wav: Cluster 1
Minor_15.wav: Cluster 1
Major_6.wav: Cluster 2
Major_0.wav: Cluster 2
Major_14.wav: Cluster 0
Minor_16.wav: Cluster 2
Minor_12.wav: Cluster 2
Major_19.wav: Cluster 3
Major_16.wav: Cluster 4
Major_4.wav: Cluster 4
Major_20.wav: Cluster 2
Minor_13.wav: Cluster 3
Minor_4.wav: Cluster 0
Minor_20.wav: Cluster 0
Major_1.wav: Cluster 1
Minor_2.wav: Cluster 4
Minor_3.wav: Cluster 1
Major_3.wav: Cluster 1
Major_7.wav: Cluster 1
Minor_17.wav: Cluster 1
Minor_1.wav: Cluster 3
Major_15.wav: Cluster 3
Minor_7.wav: Cluster 1
Major_18.wav: Cluster 4
Minor_19.wav: Cluster 3
Minor_14.wav: Cluster 0
Major_13.wav: Cluster 1
Minor_11.wav: Cluster 3
Major_5.wav: Cluster 1
Major_10.wav: Cluster 0
Minor_0.wav: Cluster 2
Major_2.wav: Cluster 0
Minor_5.wav: Cluster 3
Major_8.wav: Cluster 2
Minor_9.wav: Cluster 3
Minor_6.wav: Cluster 4
Major_17.wav: Cluster 3
Minor_8.wav: Cluster 2
Major_9.wav: Cluster 1
Minor_10.wav: Cluster 4
